In [1]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

## Submission

In [2]:
from typing import List, Tuple

import hydra
import lightning as pl
from omegaconf import DictConfig
from lightning import Callback, LightningDataModule, LightningModule, Trainer
from lightning.pytorch.loggers import Logger

from terralearn import utils

import torch
import hydra
import omegaconf
import pyrootutils
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
import timm
import matplotlib.pyplot as plt

trait_columns = [
    "X4_mean",
    "X11_mean",
    "X18_mean",
    "X50_mean",
    "X26_mean",
    "X3112_mean",
]
from sklearn.metrics import mean_squared_error, r2_score
from torchmetrics.regression import R2Score
from fgvc.models.plant_traits_model import *
from torchmetrics.functional import r2_score

In [3]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/data/plant_traits_data.yaml")
cfg.batch_size = 128
datamodule: LightningDataModule = hydra.utils.instantiate(cfg)
datamodule.setup()

In [4]:
(len(datamodule.data_train)//(64*4))*30

5040

In [5]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/model/plant_traits_model.yaml")
model = hydra.utils.instantiate(cfg)

device = 'cuda'
model = model.load_from_checkpoint("/home/ubuntu/FGVC11/logs/train/runs/dinov2_2_blocks_img_tabular/checkpoints/epoch_046.ckpt");
model = model.eval();
model = model.to(device);

/home/ubuntu/miniconda3/envs/transformers/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


In [6]:
??model.model.body.forward_head

Signature: model.model.body.forward_head(x: torch.Tensor, pre_logits: bool = False) -> torch.Tensor
Docstring: <no docstring>
Source:   
    def forward_head(self, x: torch.Tensor, pre_logits: bool = False) -> torch.Tensor:
        if self.attn_pool is not None:
            x = self.attn_pool(x)
        elif self.global_pool == 'avg':
            x = x[:, self.num_prefix_tokens:].mean(dim=1)
        elif self.global_pool:
            x = x[:, 0]  # class token
        x = self.fc_norm(x)
        x = self.head_drop(x)
        return x if pre_logits else self.head(x)
File:      ~/miniconda3/envs/transformers/lib/python3.10/site-packages/timm/models/vision_transformer.py
Type:      method

In [ ]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)


### Sanity test on validation

In [ ]:
cfg = omegaconf.OmegaConf.load("/home/ubuntu/FGVC11/configs/model/plant_traits_model.yaml")
model = hydra.utils.instantiate(cfg)
model = model.eval();
model = model.to(device);

In [ ]:
all_predictions = []
all_tgts = []
i = 0
for batch in tqdm(datamodule.val_dataloader()):
    # Unpack the batch
    x, x_ = batch["image"].to(device), batch["metadata"].to(device)
    all_tgts.append(batch["label"])
    # Move data to the device
    x, x_ = x.to(device), x_.to(device)
    break
#     # Make predictions|
#     with torch.no_grad():
#         pred_enc = model.model(x)
#         pred = model.model.le.inverse_transform(pred_enc.clone().detach())
#         pred = pred.cpu()
#     # break

#     # Append predictions to the list
#     all_predictions.append(pred)
#     i+=1
#     if i == 1:
#         break
#     # break
# # Concatenate predictions from all batches
# all_predictions = torch.concat(all_predictions, dim=0)
# all_tgts = torch.concat(all_tgts, dim=0)

In [ ]:
with torch.no_grad():
    model.model.forward_alt(x, x_)

In [ ]:
with torch.no_grad():
    a = model.model.body.forward(x)
    b = model.model.body.forward_features(x)
    c = model.model.body.forward_head(b, pre_logits=True)
        

In [ ]:
a.shape, b.shape, c.shape

In [ ]:
model.model.body.head

In [ ]:
r2_score(all_predictions, all_tgts)

In [ ]:
all_predictions.shape, all_tgts.shape

In [ ]:
r2 = TCR2Score()
r2(all_predictions ,all_tgts)

### On test data

In [ ]:
df_test = pd.read_csv('/home/ubuntu/FGVC11/data/PlantTrait/test.csv')
df_test['path'] = '/home/ubuntu/FGVC11/data/PlantTrait/test_images/' + df_test['id'].astype(str) + '.jpeg'

In [ ]:
all_predictions = []
all_tgts = []
i = 0
for batch in tqdm(datamodule.test_dataloader()):
    # Unpack the batch
    x, x_ = batch["image"].to(device), batch["metadata"].to(device)
    # Move data to the device

    # Make predictions
    with torch.no_grad():
        pred_enc = model.model.forward_alt(x, x_)
        pred = model.model.le.inverse_transform(pred_enc.clone().detach())
        pred = pred.cpu()
    # break

    # Append predictions to the list
    all_predictions.append(pred)
    # i+=1
    # if i == 1:
    #     break
    # break
# Concatenate predictions from all batches
all_predictions = torch.concat(all_predictions, dim=0)
# all_tgts = torch.concat(all_tgts, dim=0)

In [ ]:
all_predictions = all_predictions.numpy()

In [ ]:
# Create a DataFrame with the predictions and corresponding IDs
submission_df = pd.DataFrame({
    'id': df_test['id'].values,
    'X4': all_predictions[:, 0],
    'X11': all_predictions[:, 1],
    'X18': all_predictions[:, 2],
    'X50': all_predictions[:, 3],
    'X26': all_predictions[:, 4],
    'X3112': all_predictions[:, 5],
})
# submission_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission_good.csv')
r2_score(torch.tensor(sub[sub.columns[1:]].values), torch.tensor(submission_df[sub.columns[1:]].values))

In [ ]:
submission_df.to_csv('submission_df.csv', index=False)

In [ ]:
# !kaggle competitions submit -c planttraits2024 -f submission_df.csv -m "dino"

### Mapping to the closest values

In [ ]:
full_df = pd.read_csv('/home/ubuntu/FGVC11/data/PlantTrait/df_processed.csv')

In [ ]:
species_traits = full_df[trait_columns + ['species']][full_df.split != 'test'].groupby('species').mean()
species_traits.reset_index(inplace=True)

In [ ]:
mapped_traits = []
for _, row in tqdm(submission_df.iterrows(), total=len(submission_df)):
    obj_traits = []
    for col in trait_columns:
        col_sub = col.replace("_mean", "")
        pred_trt = row[col_sub]
        tgt_trts = species_traits[col]
        mapped_trait = tgt_trts[np.argmin(abs(pred_trt - tgt_trts))]
        obj_traits.append(mapped_trait)
        # break
    obj_traits = np.array(obj_traits)
    mapped_traits.append(obj_traits)

In [ ]:
mapped_traits = np.array(mapped_traits)

In [ ]:
# Create a DataFrame with the predictions and corresponding IDs
mapped_sub = pd.DataFrame({
    'id': sub['id'].values,
    'X4': mapped_traits[:, 0],
    'X11': mapped_traits[:, 1],
    'X18': mapped_traits[:, 2],
    'X50': mapped_traits[:, 3],
    'X26': mapped_traits[:, 4],
    'X3112': mapped_traits[:, 5],
})

In [ ]:
# for col in trait_columns:
#     col = col.replace("_mean", "")
#     print(f"r2 {col}: {r2_score(sub[col], mapped_sub[col])}")

r2_score(torch.tensor(sub[sub.columns[1:]].values), torch.tensor(mapped_sub[sub.columns[1:]].values))

In [ ]:
mapped_sub.to_csv('mapped_sub.csv', index=False)

In [ ]:
!kaggle competitions submit -c planttraits2024 -f mapped_sub.csv -m "dino image + tabular mapped to best"